# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd

data = pd.read_csv('homepage_actions.csv')

data.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [3]:
data['action'].value_counts()

action
view     6328
click    1860
Name: count, dtype: int64

In [4]:
data['group'].value_counts()

group
control       4264
experiment    3924
Name: count, dtype: int64

In [5]:
data['id'].value_counts()

id
750053    2
389447    2
283040    2
918794    2
208714    2
         ..
467818    1
629924    1
820336    1
424916    1
461199    1
Name: count, Length: 6328, dtype: int64

In [6]:
data[(data['id']==208714)]

,timestamp,id,group,action
6787,2016-12-30 11:11:54.407711,208714,experiment,view
6788,2016-12-30 11:12:59.433387,208714,experiment,click


In [7]:
## People whould clicked also viewed. 

In [8]:
exp_view = data[(data['group'] == 'experiment') & (data['action'] == 'view')]
exp_click = data[(data['group'] == 'experiment') & (data['action'] == 'click')]

control_view = data[(data['group'] == 'control') & (data['action'] == 'view')]
control_click = data[(data['group'] == 'control') & (data['action'] == 'click')]

In [9]:
print(f'Number of people who clicked from control group: {len(control_click)}. Number of people who clicked from experimental group: {len(exp_click)}')

Number of people who clicked from control group: 932. Number of people who clicked from experimental group: 928


In [10]:
print(f'Total number of viewers: {len(exp_view) + len(control_view)}. The total number of clickers: {len(exp_click)+len(control_click)}')

Total number of viewers: 6328. The total number of clickers: 1860


In [11]:
print(f'Number of people who viewed but did not click: {len(exp_view) + len(control_view) - len(exp_click) - len(control_click)}')

Number of people who viewed but did not click: 4468


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [12]:
#Your code here
#add count if clicked = 1, viewed = 0.

data['count'] = 1
data.head()

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1


In [13]:
control = data[data.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = data[data.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)


In [14]:
control.head()

action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0


In [15]:
import numpy as np
import scipy.stats as stats

control_click = control['click']
experiment_click = experiment['click']

/Users/michaelkearns/anaconda3/envs/my-learn-env/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [21]:
tstatistic, pvalue = stats.ttest_ind(control_click, experiment_click, equal_var=False)
print(tstatistic,pvalue)

-2.615440020788211 0.008932805628674201


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [25]:
#Your code here
control_click_rate = control_click.mean()
experiment_expected_click = control_click_rate*len(experiment_click)

print(f'The control group click rate was {control_click_rate}')
print(f'The expected number of clicks for the experiment group is {experiment_expected_click}')
print(f'The actual number of clicks for the experiment group was {experiment_click.sum()}')

The control group click rate was 0.2797118847539016
The expected number of clicks for the experiment group is 838.0168067226891
The actual number of clicks for the experiment group was 928.0


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [27]:
#Your code here
n = len(experiment_click)
p = control_click_rate
var = n*p*(1-p)
std = np.sqrt(var)
print(std)

24.568547907005815


In [29]:
z_score = (experiment_click.sum() - experiment_expected_click)/std
print(z_score)

3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [31]:
#Your code here
p_value = stats.norm.sf(z_score) 
print(p_value)

0.00012486528006951198


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: Yes. The verification steps had a smaller p-value than my initial analysis, but this helps confirm that the data is statistically significant and the null hypothesis can be rejected.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.